In [2]:
import torch
import math
from einops import rearrange,einsum

In [3]:
torch.manual_seed(4)
X = torch.randn(4, 12, 64)
positions = torch.arange(12,dtype=torch.float)

In [4]:
X_first = X[0,0]
X_first

tensor([-0.9414,  1.2632, -0.1838,  0.1505,  0.1075, -0.2780, -2.6021,  0.6245,
        -0.8684, -0.2051,  0.3976,  0.6699, -0.0537,  0.0467, -1.7671, -2.1205,
         1.5191, -0.6682,  0.0031, -0.1535,  1.1396, -0.2302,  1.1877,  0.7677,
        -0.7588, -0.1853, -0.8558, -0.2346, -0.4215,  0.8488, -0.6776, -0.9445,
        -0.4815,  1.2434,  2.3693,  0.2829, -0.2345,  1.6892,  0.2716, -0.1365,
        -0.6948, -1.3186, -0.9694,  0.6403,  0.8201, -0.9151, -2.1437,  1.4072,
        -0.0263,  2.7204, -0.5955,  0.9871,  1.0861,  0.0610,  0.0417,  0.6783,
        -0.8952, -1.0143, -0.2429, -1.5727,  1.3940, -0.1941,  0.0048, -1.3165])

In [5]:
X_second = X[0,1]
X_second,X_second.shape

(tensor([ 0.0204, -0.1652,  0.2109,  0.5167,  0.2430, -0.3522, -0.7664,  0.2973,
          1.2710,  1.8117,  1.5320, -0.2602, -1.1611, -1.2380,  1.0533,  0.2002,
          0.8206,  2.5164, -0.9537, -0.0167,  0.2526,  0.6762,  1.7228, -0.4043,
          1.0915,  0.4550, -1.2863, -0.4498, -1.0736, -1.0060, -0.6573,  1.5234,
          0.0366, -1.3862,  2.8345, -0.1750, -0.6939, -1.6809, -0.5738, -1.0947,
          0.1618, -0.9002,  0.1833,  0.5112,  0.2988,  1.5620, -2.0235, -0.0729,
         -0.3918,  0.1269, -0.8301, -1.6481,  0.2348, -0.5803, -1.5085,  0.1108,
         -0.2659,  1.0877, -0.9446, -0.7943, -1.2030,  0.6718,  0.3122, -1.6153]),
 torch.Size([64]))

In [6]:
d_k = 64
theta = 10000.0
k = torch.arange(0, d_k // 2)
R = 1.0 / (theta ** (2 * k / d_k))
R.shape

torch.Size([32])

In [7]:
angles = 1 *  R  # position is a scalar or tensor
R_cos = torch.cos(angles)
R_sin = torch.sin(angles)
angles[0],R_cos[0],R_sin[0]

(tensor(1.), tensor(0.5403), tensor(0.8415))

In [8]:
rotation_matrix = torch.eye(d_k)
for i in range(d_k // 2):
    cos_val = R_cos[i]
    sin_val = R_sin[i]
    rotation_matrix[2*i, 2*i] = cos_val
    rotation_matrix[2*i, 2*i+1] = -sin_val
    rotation_matrix[2*i+1, 2*i] = sin_val
    rotation_matrix[2*i+1, 2*i+1] = cos_val

In [12]:
rotation_matrix[:2,:]

tensor([[ 0.5403, -0.8415,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.5403,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, 

In [10]:
0.1275,0.150027

(0.1275, 0.150027)

In [11]:
X_second @ rotation_matrix.T

tensor([ 0.1500, -0.0721, -0.1978,  0.5219,  0.3934, -0.1684, -0.8209, -0.0424,
         0.6446,  2.1171,  1.5502,  0.1070, -0.9238, -1.4239,  1.0173,  0.3385,
         0.5653,  2.5857, -0.9497, -0.0881,  0.2142,  0.6893,  1.7383, -0.3313,
         1.0766,  0.4893, -1.2753, -0.4801, -1.0555, -1.0249, -0.6775,  1.5145,
         0.0504, -1.3858,  2.8358, -0.1538, -0.6845, -1.6847, -0.5692, -1.0971,
         0.1646, -0.8997,  0.1821,  0.5116,  0.2960,  1.5625, -2.0234, -0.0756,
        -0.3919,  0.1265, -0.8289, -1.6487,  0.2351, -0.5801, -1.5086,  0.1102,
        -0.2663,  1.0876, -0.9444, -0.7945, -1.2031,  0.6716,  0.3124, -1.6153])